In [10]:
! pip install pytorch-lightning==2.1.2
! pip3 install numpy==1.23.5
! pip install protobuf==3.20.*
! pip install onnx
! pip install wandb

In [11]:
import os
from datetime import datetime

import wandb
import torch
import pytorch_lightning as pl

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
import torchmetrics

from model import DenseNet, Model, DataModule

In [12]:
torch.set_float32_matmul_precision('high')

# Data

In [13]:
#TODO: Hydra
#Train/Val
train_dataset_name = 'div2k'
train_dataset_path = '/gsn/datasets/train/COCO25K'
val_dataset_path = '/gsn/datasets/train/COCO25K'

#Test
test_dataset_name = 'BSD100'
test_dataset_path_hr = '/gsn/datasets/test/BSD100/image_SRF_4/HR'
test_dataset_path_lr = '/gsn/datasets/test/BSD100/image_SRF_4/HR'

# Train

In [14]:
#TODO: Hydra
IMAGE_SIZE = 100
SCALING_FACTOR = 4

NUM_EPOCHS = 4
LEARNING_RATE = 1e-3
BATCH_SIZE = 200
NUM_WORKERS = 4

In [15]:
! wandb login --relogin 88a995e9bf3cca3f7e93c8c48c2bb5835f08305a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
os.environ['WANDB_NOTEBOOK_NAME'] = '/gsn/main'

In [17]:
MODEL_CKPT_PATH = 'model/'
MODEL_CKPT = 'model-{epoch:02d}-{val_loss:.2f}'

checkpoint_callback = ModelCheckpoint(
    monitor='train_loss', #change to val_loss
    dirpath=MODEL_CKPT_PATH,
    filename=MODEL_CKPT,
    save_top_k=3,
    mode='min')

In [18]:
# Inicjalizacja wandb
now = datetime.now()
run_name = "train_" + now.strftime('%Y-%m-%d_%H:%M:%S')
tags = ["train"]
run = wandb.init(
    project='gsn_super_resolution',
    name=run_name,
    tags=tags,
    config={
        "dataset": train_dataset_name,
        "epochs": NUM_EPOCHS,
        "scaling_factor": SCALING_FACTOR,
        "image_size": IMAGE_SIZE,
    },
)
wandb_logger = WandbLogger(project='gsn_super_resolution')

# Inicjalizacja modelu
dm =  DataModule(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, 
                 image_size=IMAGE_SIZE, scaling_factor=SCALING_FACTOR,
                 train_dataset_path=train_dataset_path,
                 val_dataset_path=val_dataset_path,
                 test_dataset_path=train_dataset_path)

dense_net = DenseNet()
model = Model(model=dense_net, scaling_factor=SCALING_FACTOR, learning_rate=LEARNING_RATE)

# Inicjalizacja trenera
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, logger=wandb_logger, callbacks=[checkpoint_callback], log_every_n_steps=2, fast_dev_run=False)

# Trenowanie modelu
trainer.fit(model,datamodule=dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | DenseNet | 1.0 M 
-----------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.164     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


In [19]:
artifact = wandb.Artifact('model', type='model')
artifact.add_dir(MODEL_CKPT_PATH)

run.log_artifact(artifact)
run.join()
wandb.finish()

wandb: Adding directory to artifact (./model)... Done. 0.0s
wandb: WARNING wandb.run.join() is deprecated, please use wandb.run.finish().


epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▆▆▆▆▆▆▆▆▆▆██████████
train_loss_epoch,█▁▁▁
train_loss_step,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▂▂▁▁▁▂▂▂▃▃▃▄▃▃▃▄▄▄▄▅▅▆▆▅▅▅▆▆▆▆▇█▆▇▇▇▇██
val_loss_epoch,█▄▂▁
val_loss_step,▆▄▅▆█▅▄▅▅▃▅▅▃▃▂▃▃▅▂▅▂▂▁▂▂▂▃▂▄▃▁▄▄▁▂▄▂▂▂▂
epoch,3
train_loss_epoch,0.00621
train_loss_step,0.0058
trainer/global_step,499
val_loss_epoch,0.00604


# Test

In [20]:
# Inicjalizacja wandb
now = datetime.now()
run_name = "test_" + now.strftime('%Y-%m-%d_%H:%M:%S')
tags = ["test"]
run = wandb.init(
    project='gsn_super_resolution',
    name=run_name,
    tags=tags,
    config={
        "dataset": test_dataset_name,
        "scaling_factor": SCALING_FACTOR,
        "image_size": IMAGE_SIZE,
    },
)
wandb_logger = WandbLogger(project='gsn_super_resolution')

trainer = pl.Trainer(logger=wandb_logger, log_every_n_steps=2)

# Test
trainer.test(model=model, datamodule=dm)

# Zamknięcie wandb
wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.006040374748408794
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_loss,0.00604
trainer/global_step,0


# Archive model

In [21]:
model_archive_path = '/gsn/models_archive/'
if not os.path.exists(model_archive_path):
   os.makedirs(model_archive_path)

file_path = model_archive_path + "model_%s.onnx" % now.strftime('%Y-%m-%d_%H:%M:%S')

input_sample = torch.randn((1, 1, IMAGE_SIZE, IMAGE_SIZE))
model.to_onnx(file_path, input_sample, export_params=True)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

